### Import Required Libraries

In [ ]:
import pandas as pd
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import col


### Connecting to Snowflake

In [ ]:
# Creating a Snowflake session inside the notebook
session = Session.builder.getOrCreate()


### Fetch Data from Snowflake Table

In [ ]:
# Read customer reviews table from Snowflake
customer_reviews_df = session.table("MARKETINGANALYTICS.CUSTOMER.CUSTOMER_REVIEWS").to_pandas()

### Performing Sentiment Analysis

In [ ]:
# Initialize Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# Function to calculate sentiment score
def calculate_sentiment(review):
    sentiment = sia.polarity_scores(review)
    return sentiment['compound']

# Function to categorize sentiment based on score and rating
def categorize_sentiment(score, rating):
    if score > 0.05:
        return 'Positive' if rating >= 4 else 'Mixed Positive' if rating == 3 else 'Mixed Negative'
    elif score < -0.05:
        return 'Negative' if rating <= 2 else 'Mixed Negative' if rating == 3 else 'Mixed Positive'
    else:
        return 'Neutral' if rating == 3 else 'Positive' if rating >= 4 else 'Negative'

# Function to bucket sentiment scores
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'
    else:
        return '-1.0 to -0.5'


### Apply Sentiment Analysis to DataFrame

In [ ]:
# Apply sentiment analysis
customer_reviews_df['SentimentScore'] = customer_reviews_df['REVIEWTEXT'].apply(calculate_sentiment)

# Apply sentiment categorization
customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['RATING']), axis=1)

# Apply sentiment bucketing
customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)


### Store the Processed Data Back into Snowflake

In [ ]:
# Convert Pandas DataFrame back to Snowpark DataFrame
snowpark_df = session.create_dataframe(customer_reviews_df)

# Save the transformed data as a new table
snowpark_df.write.mode("overwrite").save_as_table("customer_reviews_with_sentiment")
